# YOUR PROJECT TITLE

> **Note the following:** 
> 1. This is *not* meant to be an example of an actual **data analysis project**, just an example of how to structure such a project.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `dataproject.py` file includes a function which can be used multiple times in this notebook.

Imports and set magics:

In [20]:
import pandas as pd
import numpy as np
import datetime

import matplotlib.pyplot as plt
plt.rcParams.update({"axes.grid":True,"grid.color":"black","grid.alpha":"0.25","grid.linestyle":"--"})
plt.rcParams.update({'font.size': 14})
import ipywidgets as widgets

from dstapi import DstApi # install with `pip install git+https://github.com/alemartinello/dstapi`

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# user written modules
import dataproject

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Read and clean data

In [21]:
data = DstApi('EJ12') 

Import your data, either through an API or manually, and load it. 

## Explore each data set

In [31]:
tabsum = data.tablesummary(language='en')
display(tabsum)

Table EJ12: Seasonally adjusted sales of property by region, property category, unit and time
Last update: 2023-03-31T08:00:00


,variable name,# values,First value,First value label,Last value,Last value label,Time variable
0,REGION,6,000,All Denmark,081,Region Nordjylland,False
1,EJKAT20,3,0111,One-family houses,2103,"Owner-occupied flats, total",False
2,TAL,6,202,"Priceindex, seasonally adjusted",370,"Number of sales, percentage change compared to...",False
3,Tid,72,2005K1,2005Q1,2022K4,2022Q4,True


In [36]:
# The available values for a each variable: 
for variable in tabsum['variable name']:
    print(variable+':')
    display(data.variable_levels(variable, language='en'))

REGION:


,id,text
0,000,All Denmark
1,084,Region Hovedstaden
2,085,Region Sjælland
3,083,Region Syddanmark
4,082,Region Midtjylland
5,081,Region Nordjylland


EJKAT20:


,id,text
0,0111,One-family houses
1,0801,Weekend cottages
2,2103,"Owner-occupied flats, total"


TAL:


,id,text
0,202,"Priceindex, seasonally adjusted"
1,362,"Priceindex, percentage change compared to prev..."
2,364,"Priceindex, percentage change compared to same..."
3,204,"Number of sales, seasonally adjusted"
4,368,"Number of sales, percentage change compared to..."
5,370,"Number of sales, percentage change compared to..."


Tid:


,id,text
0,2005K1,2005Q1
1,2005K2,2005Q2
2,2005K3,2005Q3
3,2005K4,2005Q4
4,2006K1,2006Q1
...,...,...
67,2021K4,2021Q4
68,2022K1,2022Q1
69,2022K2,2022Q2
70,2022K3,2022Q3


**Skriv at vi kan se at datasættet indeholder x, y z som vi ikke er interesserede i bla bla bla**

In [37]:
params = data._define_base_params(language='en')
params

{'table': 'ej12',
 'format': 'BULK',
 'lang': 'en',
 'variables': [{'code': 'REGION', 'values': ['*']},
  {'code': 'EJKAT20', 'values': ['*']},
  {'code': 'TAL', 'values': ['*']},
  {'code': 'Tid', 'values': ['*']}]}

In [45]:
params = {'table': 'ej12',
 'format': 'BULK',
 'lang': 'en',
 'variables': [{'code': 'REGION', 'values': ['*']},
  {'code': 'EJKAT20', 'values': ['*']},
  {'code': 'TAL', 'values': ['202', '362']},
  {'code': 'Tid', 'values': ['*']}]}

In [46]:
sales_api = data.get_data(params=params)
sales_api.sort_values(by=['REGION', 'TID', 'EJKAT20'], inplace=True)
sales_api.head(10)

,REGION,EJKAT20,TAL,TID,INDHOLD
1976,All Denmark,One-family houses,"Priceindex, seasonally adjusted",2005Q1,75.9
1977,All Denmark,One-family houses,"Priceindex, percentage change compared to prev...",2005Q1,..
1980,All Denmark,"Owner-occupied flats, total","Priceindex, seasonally adjusted",2005Q1,73.2
1981,All Denmark,"Owner-occupied flats, total","Priceindex, percentage change compared to prev...",2005Q1,..
1978,All Denmark,Weekend cottages,"Priceindex, seasonally adjusted",2005Q1,78.6
1979,All Denmark,Weekend cottages,"Priceindex, percentage change compared to prev...",2005Q1,..
2232,All Denmark,One-family houses,"Priceindex, seasonally adjusted",2005Q2,79.3
2233,All Denmark,One-family houses,"Priceindex, percentage change compared to prev...",2005Q2,4.5
2236,All Denmark,"Owner-occupied flats, total","Priceindex, seasonally adjusted",2005Q2,78.1
2237,All Denmark,"Owner-occupied flats, total","Priceindex, percentage change compared to prev...",2005Q2,6.7


In order to be able to **explore the raw data**, you may provide **static** and **interactive plots** to show important developments 

**Interactive plot** :

In [27]:
def plot_func():
    # Function that operates on data set
    pass

widgets.interact(plot_func, 
    # Let the widget interact with data through plot_func()    
); 


interactive(children=(Output(),), _dom_classes=('widget-interact',))

Explain what you see when moving elements of the interactive plot around. 

# Merge data sets

# Analysis

To get a quick overview of the data, we show some **summary statistics** on a meaningful aggregation. 

MAKE FURTHER ANALYSIS. EXPLAIN THE CODE BRIEFLY AND SUMMARIZE THE RESULTS.

# Conclusion

ADD CONCISE CONLUSION.